## 📚 Step 1: Import Everything

This imports all the functions we need to run this notebook.

In [1]:
from prompt_runner import (
    load_environment,
    load_configuration, 
    load_dataset,
    tasks_to_dataframe,
    run_all_tasks,
    results_summary_dataframe,
    save_results,
    create_results_table
)
import os
import pandas as pd
from litellm import supports_response_schema
pd.set_option('display.max_colwidth', None)


print("✅ Imports complete!")

✅ Imports complete!


## 🔍 Step 2: Check Which Models Support Structured Output

Before we start, let's see which models work with our Pydantic structured output. Using structured output is a way of making sure that the models give us the output in the format we want, like only answering "positive" or negative". 
This helps you choose the right models for your config.json file, which is how we control what models we're using.

In [2]:
print("🔍 Checking model support for structured output (Pydantic)...")
# Test models for json_schema support (what we actually need)
test_models = [
    ("gpt-4o-mini", "openai"),
    ("gpt-4o", "openai"), 
    ("gpt-3.5-turbo", "openai"),
    ("claude-3-haiku-20240307", "anthropic"),
    ("claude-3-5-sonnet-20241022", "anthropic"),
]

supported = []
for model, provider in test_models:
    try:
        # Use the correct function for checking Pydantic/json_schema support
        has_support = supports_response_schema(model=model, custom_llm_provider=provider)
        if has_support:
            supported.append(f"{model} ({provider})")
            print(f"✅ {model} - Supports Pydantic structured output")
        else:
            print(f"❌ {model} - No Pydantic structured output support")
    except Exception as e:
        print(f"❓ {model} - Could not check: {str(e)}")
        
print(f"\n💡 Supported models for config.json: {supported}")

🔍 Checking model support for structured output (Pydantic)...
✅ gpt-4o-mini - Supports Pydantic structured output
✅ gpt-4o - Supports Pydantic structured output
❌ gpt-3.5-turbo - No Pydantic structured output support
✅ claude-3-haiku-20240307 - Supports Pydantic structured output
✅ claude-3-5-sonnet-20241022 - Supports Pydantic structured output

💡 Supported models for config.json: ['gpt-4o-mini (openai)', 'gpt-4o (openai)', 'claude-3-haiku-20240307 (anthropic)', 'claude-3-5-sonnet-20241022 (anthropic)']


## 🔑 Step 3: Load Environment Variables

This loads your API keys from the .env file.
Make sure you have the API keys you need for the providers you want to use.

In [3]:
load_environment()

# Show what API keys we actually found
api_keys_to_check = ["OPENAI_API_KEY", "ANTHROPIC_API_KEY", "GEMINI_API_KEY"]

for key in api_keys_to_check:
    os.getenv(key)

✓ Found 3 API key(s) in .env:
   - OPENAI_API_KEY
   - ANTHROPIC_API_KEY
   - GEMINI_API_KEY
✓ Environment variables loaded from .env


## ⚙️ Step 4: Load Configuration

This reads your config.json file which contains:
- Which prompts to test
- Which models to use
- What temperatures to try
- How to format the output

In [4]:
config = load_configuration("config.json")

print(f"\n💡 To modify your experiment:")
print(f"   📝 Edit config.json to change:")
print(f"      - prompts: Add new prompt templates")
print(f"      - models: Try different LLMs")
print(f"      - temperatures: test effect of consistency")
print(f"      - sample_size: Test on more data (currently {config['data_settings']['sample_size']})")
print(f"\n🔄 After editing config.json, restart this notebook to see changes")

✓ 3 prompts, 2 models, 2 temperatures

📝 Prompts:
   1. sentiment_basic: Classify this movie review as either positive or negative: '{text}'

   2. sentiment_instruction: Read the following movie review and determine if it's positive or negative.

Review: '{text}'

   3. sentiment_bad: hey is this good or bad idk: '{text}'


💡 To modify your experiment:
   📝 Edit config.json to change:
      - prompts: Add new prompt templates
      - models: Try different LLMs
      - temperatures: test effect of consistency
      - sample_size: Test on more data (currently 10)

🔄 After editing config.json, restart this notebook to see changes


## 📁 Step 5: Load Your Data

This loads your CSV file and samples data for testing.
It automatically balances the classes (equal positive/negative samples).

In [5]:
samples = load_dataset("IMDB Dataset.csv", config)

print(f"📊 Loaded {len(samples)} samples")
print(f"📈 Label distribution:")
label_counts = samples[config['data_settings']['label_column']].value_counts()
for label, count in label_counts.items():
    print(f"   - {label}: {count}")

# Show sample data
samples.head(3)

✓ 10 samples loaded
📊 Loaded 10 samples
📈 Label distribution:
   - positive: 5
   - negative: 5


,review,sentiment
0,"Some films just simply should not be remade. This is one of them. In and of itself it is not a bad film. But it fails to capture the flavor and the terror of the 1963 film of the same title. Liam Neeson was excellent as he always is, and most of the cast holds up, with the exception of Owen Wilson, who just did not bring the right feel to the character of Luke. But the major fault with this version is that it strayed too far from the Shirley Jackson story in it's attempts to be grandiose and lost some of the thrill of the earlier film in a trade off for snazzier special effects. Again I will say that in and of itself it is not a bad film. But you will enjoy the friction of terror in the older version much more.",positive
1,"""Ardh Satya"" is one of the finest film ever made in Indian Cinema. Directed by the great director Govind Nihalani, this one is the most successful Hard Hitting Parallel Cinema which also turned out to be a Commercial Success. Even today, Ardh Satya is an inspiration for all leading directors of India.<br /><br />The film tells the Real-life Scenario of Mumbai Police of the 70s. Unlike any Police of other cities in India, Mumbai Police encompasses a Different system altogether. Govind Nihalani creates a very practical Outlay with real life approach of Mumbai Police Environment.<br /><br />Amongst various Police officers & colleagues, the film describes the story of Anand Velankar, a young hot-blooded Cop coming from a poor family. His father is a harsh Police Constable. Anand himself suffers from his father's ideologies & incidences of his father's Atrocities on his mother. Anand's approach towards immediate action against crime, is an inert craving for his own Job satisfaction. The film is here revolved in a Plot wherein Anand's constant efforts against crime are trampled by his seniors.This leads to frustrations, as he cannot achieve the desired Job-satisfaction. Resulting from the frustrations, his anger is expressed in excessive violence in the remand rooms & bars, also turning him to an alcoholic.<br /><br />The Spirit within him is still alive, as he constantly fights the system. He is aware of the system of the Metro, where the Police & Politicians are a inertly associated by far end. His compromise towards unethical practice is negative. Finally he gets suspended.<br /><br />The Direction is a master piece & thoroughly hard core. One of the best memorable scenes is when Anand breaks in the Underworld gangster Rama Shetty's house to arrest him, followed by short conversation which is fantastic. At many scenes, the film has Hair-raising moments.<br /><br />The Practical approach of Script is a major Punch. Alcoholism, Corruption, Political Influence, Courage, Deceptions all are integral part of Mumbai police even today. Those aspects are dealt brilliantly.<br /><br />Finally, the films belongs to the One man show, Om Puri portraying Anand Velankar traversing through all his emotions absolutely brilliantly.",positive
2,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the s

## 🎯 Step 6: Create All Task Combinations

This creates every possible combination of:
- Each prompt × Each model × Each temperature × Each sample

This is where we define exactly what experiments to run!

In [7]:
tasks_df = tasks_to_dataframe(config, samples)

print(f"🎯 Total: {len(tasks_df)} tasks")
print(f"⏱️  Estimated time: {tasks_df.attrs['estimated_time_min']:.1f} minutes")

# Show sample tasks
tasks_df[['prompt_id', 'model', 'temperature', 'text']].head()

🎯 Total: 120 tasks
⏱️  Estimated time: 1.0 minutes


,prompt_id,model,temperature,text
0,sentiment_basic,gpt-4o-mini,0.0,"Some films just simply should not be remade. This is one of them. In and of itself it is not a bad film. But it fails to capture the flavor and the terror of the 1963 film of the same title. Liam Neeson was excellent as he always is, and most of the cast holds up, with the exception of Owen Wilson, who just did not bring the right feel to the character of Luke. But the major fault with this version is that it strayed too far from the Shirley Jackson story in it's attempts to be grandiose and lost some of the thrill of the earlier film in a trade off for snazzier special effects. Again I will say that in and of itself it is not a bad film. But you will enjoy the friction of terror in the older version much more."
1,sentiment_basic,gpt-4o-mini,0.0,"""Ardh Satya"" is one of the finest film ever made in Indian Cinema. Directed by the great director Govind Nihalani, this one is the most successful Hard Hitting Parallel Cinema which also turned out to be a Commercial Success. Even today, Ardh Satya is an inspiration for all leading directors of India.<br /><br />The film tells the Real-life Scenario of Mumbai Police of the 70s. Unlike any Police of other cities in India, Mumbai Police encompasses a Different system altogether. Govind Nihalani creates a very practical Outlay with real life approach of Mumbai Police Environment.<br /><br />Amongst various Police officers & colleagues, the film describes the story of Anand Velankar, a young hot-blooded Cop coming from a poor family. His father is a harsh Police Constable. Anand himself suffers from his father's ideologies & incidences of his father's Atrocities on his mother. Anand's approach towards immediate action against crime, is an inert craving for his own Job satisfaction. The film is here revolved in a Plot wherein Anand's constant efforts against crime are trampled by his seniors.This leads to frustrations, as he cannot achieve the desired Job-satisfaction. Resulting from the frustrations, his anger is expressed in excessive violence in the remand rooms & bars, also turning him to an alcoholic.<br /><br />The Spirit within him is still alive, as he constantly fights the system. He is aware of the system of the Metro, where the Police & Politicians are a inertly associated by far end. His compromise towards unethical practice is negative. Finally he gets suspended.<br /><br />The Direction is a master piece & thoroughly hard core. One of the best memorable scenes is when Anand breaks in the Underworld gangster Rama Shetty's house to arrest him, followed by short conversation which is fantastic. At many scenes, the film has Hair-raising moments.<br /><br />The Practical approach of Script is a major Punch. Alcoholism, Corruption, Political Influence, Courage, Deceptions all are integral part of Mumbai police even today. Those aspects are dealt brilliantly.<br /><br />Finally, the films belongs to the One man show, Om Puri portraying Anand Velankar traversing through all his emotions absolutely brilliantly."
2,sentiment_basic,gpt-4o-mini,0.0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dea

## 🚀 Step 7: Run All Prompt Tasks

This is where the magic happens! 
- Makes API calls to test each prompt
- Uses parallel processing to go faster
- Shows real-time progress
- Handles errors gracefully

⚠️ This will cost money (API calls) and take a few minutes!

In [8]:
print("🚀 Starting prompt execution...")
print("💰 Note: This will make API calls and cost money!")

# Convert DataFrame to tasks list for execution
tasks = tasks_df.to_dict('records')
results = run_all_tasks(tasks, config, max_workers=4)

print(f"\n🎉 Completed {len(results)} tasks!")

# Show sample results
results_df = results_summary_dataframe(results)
print(f"✅ Success rate: {results_df.attrs['success_rate']:.1f}%")

results_df[['prompt_id', 'model', 'temperature', 'true_label', 'prediction']].head()

🚀 Starting prompt execution...
💰 Note: This will make API calls and cost money!
Running 120 tasks with 4 parallel workers...
Progress: 120/120 (100%)
✓ Completed all 120 tasks

🎉 Completed 120 tasks!
✅ Success rate: 100.0%


,prompt_id,model,temperature,true_label,prediction
0,sentiment_basic,gpt-4o-mini,0.0,positive,positive
1,sentiment_basic,gpt-4o-mini,0.0,positive,negative
2,sentiment_basic,gpt-4o-mini,0.0,positive,positive
3,sentiment_basic,gpt-4o-mini,0.0,positive,positive
4,sentiment_basic,gpt-4o-mini,0.0,positive,positive


## 📈 Step 8: Quick Analysis

This gives you a fast overview of how well each combination performed.
Look for patterns:
- Which prompts work better?
- Does temperature matter?
- Are there differences between models?

In [9]:
create_results_table(results)

GT(_tbl_data=                   Prompt                    Model  Temperature  Accuracy  \
0           sentiment_bad  claude-3-haiku-20240307          0.0       0.9   
1           sentiment_bad  claude-3-haiku-20240307          0.7       0.9   
2           sentiment_bad              gpt-4o-mini          0.0       0.9   
3           sentiment_bad              gpt-4o-mini          0.7       0.9   
4         sentiment_basic  claude-3-haiku-20240307          0.0       0.9   
5         sentiment_basic  claude-3-haiku-20240307          0.7       0.9   
6         sentiment_basic              gpt-4o-mini          0.0       0.9   
7         sentiment_basic              gpt-4o-mini          0.7       0.9   
8   sentiment_instruction  claude-3-haiku-20240307          0.0       0.9   
9   sentiment_instruction  claude-3-haiku-20240307          0.7       0.9   
10  sentiment_instruction              gpt-4o-mini          0.0       0.9   
11  sentiment_instruction              gpt-4o-mini          0.7       0.9   

    Correct  Total  
0         9     10  
1         9     10  
2         9     10  
3         9     10  
4         9     10  
5         9     10  
6         9     10  
7         9     10  
8         9     10  
9         9     10  
10        9     10  
11        9     10  , _body=<great_tables._gt_data.Body object at 0x1135e86e0>, _boxhead=Boxhead([ColInfo(var='Prompt', type=<ColInfoTypeEnum.default: 1>, column_label='Prompt', column_align='left', column_width=None), ColInfo(var='Model', type=<ColInfoTypeEnum.default: 1>, column_label='Model', column_align='left', column_width=None), ColInfo(var='Temperature', type=<ColInfoTypeEnum.default: 1>, column_label='Temperature', column_align='center', column_width=None), ColInfo(var='Accuracy', type=<ColInfoTypeEnum.default: 1>, column_label='Accuracy', column_align='right', column_width=None), ColInfo(var='Correct', type=<ColInfoTypeEnum.default: 1>, column_label='Correct', column_align='center', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='Total', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x1135e82f0>, _spanners=Spanners([]), _heading=Heading(title='Prompt Testing Results', subtitle='12 combinations tested', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x1135e8ad0>, _formats=[<great_tables._gt_data.FormatInfo object at 0x1135e8980>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_bo

## 💾 Step 9: Optional: Save Results to Files

This saves your results as a CSV file.
Files are timestamped so you won't overwrite previous experiments.

In [10]:
file_path = save_results(results, output_path="experiment_results")